In [1]:
import json
import random
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
corpus = list()

In [3]:
with open ('articoli.txt', 'r', encoding = 'utf-8') as file:
    corpus = json.loads(file.read())
    
# eliminazione di quelle frasi che sono delle domande
# (per evitare che il chatbot risponda con delle domande)
for sentence in corpus:
    if (sentence[len(sentence)-1] == '?'):
        corpus.remove(sentence)

In [4]:
with open('q&a.json', 'r') as data_structure:
    q_a = json.load(data_structure)
    # appendiamo le chiavi del dizionario (le domande delle FAQ)
    for k in q_a.keys():
        corpus.append(k)

# Costruzione del modello

In [5]:
# carichiamo il modello tfidf
tfidf = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_df = 1000)
matrix_tfidf = tfidf.fit_transform(corpus)

In [6]:
# riordina la lista in ordine decrescente
def find_index(cos_sim_matrix):
    top, index = 0.0, -1
    i = 0
    for i in range(len(cos_sim_matrix)):
        if cos_sim_matrix[i] > top:
            top = cos_sim_matrix[i]
            index = i
    return index

In [7]:
def compute_similarity (model, matrix, query):
    vec_query = model.transform([query])
    # calcoliamo similarità tra query e count matrix
    # e rendiamo la similarity matrix unidimensionale
    cos_sim = cosine_similarity(vec_query, matrix).flatten()
    
    return cos_sim

In [8]:
# function to return a random greeting responde to a users greeting
def greeting_response(text):
    text = text.lower()
    
    # bots greeting response
    bot_greetings = ['ciao', 'ehi']
    # user's greeting
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [9]:
def bot_response(user_input, model, matrix, q_a):
    bot_response = ''
    
    # cosine similarity function calcola la similarità tra
    # X e Y. Nel nostro caso X = query (vvero domanda 
    # dell'utente) e matrix di similarità calcolata in fase di
    # training del modello
    similarity_scores_list = compute_similarity(model, matrix, user_input)
    # lista di indici riodinati a seconda del similarity score
    index = find_index(similarity_scores_list)
     
    if index == -1:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'
        return bot_response
    else:
        if corpus[index] in q_a.keys():
            bot_response = bot_response + q_a[corpus[index]][0]
        # altrimenti, prende una frase degli articoli
        else:
            bot_response = bot_response + corpus[index]
            response_flag = 1 # abbiamo trovato la risposta
        return bot_response

In [10]:
def chatbot_response(text):
    exit_list = ['esci','stop']
    user_input = text
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        #break
    else:
        if greeting_response(user_input) != None:
            return greeting_response(user_input)
        else:
            
            return bot_response(user_input, tfidf, matrix_tfidf, q_a)

# Implementazione interfaccia

In [11]:
def send():
    te = messageWindow.get("1.0",END)
    chatWindow.insert(INSERT, 'Me: '+ '\n')
    chatWindow.config(foreground="#442265", font=("Verdana", 14 ))
    chatWindow.insert(INSERT,te)
    chatWindow.insert(INSERT,'\n')
    chatWindow.insert(INSERT, 'chatbot: '+ '\n\n')
    c = chatbot_response(te)
    chatWindow.insert(INSERT,c)
    chatWindow.insert(INSERT,'\n\n')

In [12]:
from tkinter import *

root = Tk()

root.title("ChatBot")
root.geometry("400x500")
root.resizable(width=TRUE, height=TRUE)

chatWindow = Text(root, bd=1, bg="light blue",  width="50", height="8", font=("Arial", 17), foreground="black")
chatWindow.place(x=6,y=6, height=386, width=370)


messageWindow = Text(root, bd=0, bg="light blue",width="30", height="4", font=("Arial", 17), foreground="black")
messageWindow.place(x=128, y=400, height=88, width=260)

scrollbar = Scrollbar(root, command=chatWindow.yview)
scrollbar.place(x=375,y=5, height=385)

Button= Button(root, text="Send",  width="12", height=5,
                    bd=0, bg="red", activebackground="blue",foreground='red',font=("Arial", 12), command = send)
Button.place(x=6, y=400, height=88)


root.mainloop()